In [1]:
import os

import gmsh
import numpy as np

import commons, utils

In [2]:
LX = 40e-6
LY = 40e-6
LZ = 150e-6
LSEP = 75e-6
LCAT = 75e-6
Rp = 10e-6
workdir = "output/reaction_distribution/40-40-150/"
utils.make_dir_if_missing(workdir)
output_msh_path = os.path.join(workdir, "mesh.msh")
markers = commons.Markers()

In [3]:
gmsh.initialize()
gmsh.model.add('geo')
box_left = gmsh.model.occ.addBox(0, 0, 0, LX, LY, LZ)
box_right = gmsh.model.occ.addBox(0, 0, LZ - Rp, LX, LY, Rp)
cyl = gmsh.model.occ.addCylinder(0.5 * LX, 0.5 * LY, LSEP, 0, 0, LCAT - Rp, Rp)
merged = gmsh.model.occ.fuse([(3, box_right)], [(3, cyl)])
gmsh.model.occ.cut([(3, box_left)], merged[0], removeTool=False)

gmsh.model.occ.synchronize()
vols = gmsh.model.get_entities(3)
gmsh.model.addPhysicalGroup(3, [vols[0][1]], markers.electrolyte, "electrolyte")
gmsh.model.addPhysicalGroup(3, [vols[1][1]], markers.positive_am, "positive_am")

surfs = gmsh.model.get_entities(2)
left = []
right = []
interface = []
insulated_se = []
insulated_am = []
for _, surf in surfs:
    com = gmsh.model.occ.get_center_of_mass(2, surf)
    if np.isclose(com[2], 0):
        left.append(surf)
    elif np.isclose(com[2], LZ):
        right.append(surf)
    elif np.isclose(com[2], 0.5*(LZ - Rp)):
        insulated_se.append(surf)
    elif np.isclose(com[2], LZ - 0.5 * Rp):
        insulated_am.append(surf)
    else:
        interface.append(surf)
gmsh.model.addPhysicalGroup(2, left, markers.left, "left")
gmsh.model.addPhysicalGroup(2, insulated_se, markers.insulated_electrolyte, "insulated_electrolyte")
gmsh.model.addPhysicalGroup(2, right, markers.right, "right")
gmsh.model.addPhysicalGroup(2, insulated_am, markers.insulated_positive_am, "insulated_am")
gmsh.model.addPhysicalGroup(2, interface, markers.electrolyte_v_positive_am, "electrolyte_positive_am_interface")
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write(output_msh_path)
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 30%] Meshing curve 18 (Line)
Info    : [ 30%] Meshing curve 19 (Line)
Info    : [ 40%] Meshing curve 20 (Line)
Info    : [ 40%] Meshing curve 21 (Line)
Info    : [ 40%] Meshing curve 22 (Line)
Info    : [ 50%] Meshing curve 23 (Line)
Info    : [ 50%] Meshing curve 24 (Line)
Info    : [ 60%] Meshing curve 25 (Circle)
Info    : [ 60%] Meshing curve 26 (Line)
Info    : [ 70%] Meshing curve 27 (Circle)
Info    : [ 70%] Meshing curve 28 (Line)
Info    : [ 70%] Meshing curve 29 (Line)
Info    : [ 80%] Meshing curve 30 (Line)
Info    : [ 80%] Meshing curve 31 (Line)
Info    : [ 90%] Meshing curve 32 (Line)
Info    : [ 90%] Meshing curve 33 (Line)
Info    : [100%] Meshing curve 34 (Line)
Info    : [100%] Meshing curve 35 (Line)
Info    : Done meshing 1D (Wa